In [12]:
import requests
import json
import pandas as pd
import numpy as np
from scipy.special import comb
import scipy.misc
scipy.misc.comb = comb
from ecopy import diversity

In [13]:
# API endpoint
URL = 'https://api.laji.fi/v0'
# Your email
EMAIL = 'behramulukir@aalto.fi'
# Token ID
TOKEN = 'GDHiX5rLpPzfjGK2owsbJLEkWCwQoLUnmtIGPUEqh40fGxmNSpyYf56jB3WTEDaQ'

In [14]:
# How to request a token
# Just query a POST with your email. You will get the token in an email they sent you.
response = requests.post(
  url = URL + '/api-users',
  data = {
    'email': EMAIL
  })

# Check if it has been registered to the site
responseContent = json.loads(response.content.decode('utf-8'))
responseContent

{'error': {'statusCode': 422,
  'name': 'ValidationError',
  'message': 'The `APIUser` instance is not valid. Details: `email` Email already exists (value: "behramulukir@aalto.fi").',
  'details': {'context': 'APIUser',
   'codes': {'email': ['uniqueness']},
   'messages': {'email': ['Email already exists']}}}}

In [15]:
# Initial request to get the last page
response = requests.get(
  url = URL + '/warehouse/query/unit/list',
  params={
    'access_token': TOKEN,
    'page': 1,
    'pageSize': 10000, # Limit 10,000 per page
    'taxonRankId': '93671, 83190, 65513, 66530, 82625, 99327', # {Vascular plants: {Plant life forms: {Trees; Evergreen trees} } }
    'orderBy': ['gathering.eventDate.begin DESC', 'document.loadDate DESC', 'unit.taxonVerbatim ASC'] # Default sorting
  })

responseContent = response.content
results = json.loads(responseContent.decode('utf-8'))

In [16]:
lastPage = results['lastPage']

# Lists of variables
date = []
latitude = []
longitude = []
municipality = []
taxonId = []
science = []
verbatim = []

for pageIndex in range(1, lastPage + 1):
    # Request data from API and go through all the pages in the loop (approximately 8m with 12 pages and 115739 rows)
    response = requests.get(
        url = URL + '/warehouse/query/unit/list',
        params={
            'access_token': TOKEN,
            'page': pageIndex,
            'pageSize': 10000, # Limit 10,000 per page
            'taxonRankId': '93671, 83190, 65513, 66530, 82625, 99327', # {Vascular plants: {Plant life forms: {Trees; Evergreen trees} } }
            'orderBy': ['gathering.eventDate.begin DESC', 'document.loadDate DESC', 'unit.taxonVerbatim ASC'] # Default sorting
    })

    responseContent = response.content
    dataset = json.loads(responseContent.decode('utf-8'))['results']

    # Extract relevant information, sometimes they might be missing
    for x in dataset:
        # Date of collection; normally it would be "DD-MM-YYYY", sometimes it could be a range "DD-MM-YYY - DD-MM-YYYY"
        if 'displayDateTime' in x['gathering']:
            date.append(x['gathering']['displayDateTime'])
        else:
            date.append(None)

        # Latitude and Longitude
        if 'conversions' in x['gathering']:
            latitude.append(x['gathering']['conversions']['wgs84CenterPoint']['lat'])
            longitude.append(x['gathering']['conversions']['wgs84CenterPoint']['lon'])
        else:
            latitude.append(None)
            longitude.append(None)

        # Municipality
        if 'interpretations' in x['gathering']  and 'municipalityDisplayname' in x['gathering']['interpretations']:
            municipality.append(x['gathering']['interpretations']['municipalityDisplayname'])
        else:
            municipality.append(None)

        # Taxon ID and Scientific Name
        if 'unit' in x and 'linkings' in x['unit']:
            taxonId.append(x['unit']['linkings']['taxon']['id'])
            science.append(x['unit']['linkings']['taxon']['scientificName'])
        else:
            taxonId.append(None)
            science.append(None)

        # Taxon Verbatim
        if 'unit' in x and 'taxonVerbatim' in x['unit']:
            verbatim.append(x['unit']['taxonVerbatim'])
        else:
            verbatim.append(None)


In [17]:
# Merge data into a dataframe; there are a total of 115739 observations, but only 115722 rows in the dataframe. This could be due to my variables selection, e.g. there are
# some observations that have none of the listed variables (no name, no time, no place)

data = {"Date": date, "Latitude": latitude, "Longitude": longitude, "Municipality": municipality, "TaxonID": taxonId, "ScientificName": science,
        "verbatim": verbatim}
df = pd.DataFrame(data=data)
print(df)
df.to_csv("../../correlation/correlationdata.csv", index=False)

Empty DataFrame
Columns: [Date, Latitude, Longitude, Municipality, TaxonID, ScientificName, verbatim]
Index: []


OSError: Cannot save file into a non-existent directory: '../../correlation'